In [1]:
# Using THREE month's worth of world news posts
'''12-1-2019 EDA'''
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer converts a collection of text documents to a matrix of token counts
import warnings
from sklearn.decomposition import LatentDirichletAllocation as LDA # Load LDA model from SKL
from collections import Counter
import pyLDAvis # For interactive topic model visualization
from pyLDAvis import sklearn as sklearn_lda

C:\Users\Hannah\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [2]:
df = pd.read_csv('C:\\Users\\Hannah\\Desktop\\UT_Austin\\Analytics_MSBA\\Fall_19\\Data_mgmt_intro\\Final_project\\summer19_worldnews.csv')

In [3]:
df.head(3)

,body,author,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit
0,[removed],[deleted],1557015514,t5_2qh13,t3_b8k17q,t1_emi96x1,0,1561539494,0,0,emj0itm,worldnews
1,Add Australia and Canada to the trump ally lis...,Houjix,1558528774,t5_2qh13,t3_b9pc0e,t1_ek629yu,0,1563090093,0,0,eofaisv,worldnews
2,"So, I just noticed your reply, and it worries ...",epistemic_zoop,1560687132,t5_2qh13,t3_b9wu6k,t1_ek90vw5,0,1570082531,0,0,erbmu65,worldnews


In [4]:
# Clean data

# Remove null comments
df = df[pd.notnull(df['body'])]

# Convert all posts to string
df['body'] = df['body'].astype(str)

# Remove duplicates
df = df.drop_duplicates()

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
# Clean data

# Remove stopwords
df['body_sw'] = df['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Remove special characters, make all characters lowercase
df['body_sw_p'] = df['body_sw'].str.replace('[^\w\s]', '').str.lower()

# Remove other common words
mask = df['body_sw_p'].str.contains('deleted','people')
df=df[~mask]

<>:7: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<ipython-input-6-26edbc285c0a>:7: DeprecationWarning: invalid escape sequence \w
  df['body_sw_p'] = df['body_sw'].str.replace('[^\w\s]', '').str.lower()


In [7]:
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the processed titles to get matrix of token counts
count_data_tokens = count_vectorizer.fit_transform(df['body_sw_p'])

In [8]:
warnings.simplefilter("ignore", DeprecationWarning)

# Function to print topics and the top 10 labels in each
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names() #words in corpus
    for topic_idx, topic in enumerate(model.components_): #enumerate adds a counter to an iterable
        print("\nTopic #%d:" % topic_idx) #%d is used as a placeholder for numeric or decimal values
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) #numpy.argsort() returns the indices that would sort an array

In [9]:
# Tweak the two parameters below
number_topics = 20
number_words = 10

# Create and fit the LDA model
# n_jobs specifyies the maximum number of concurrently running jobs. If set to -1, all CPUs are used. 
# n_components number of features which a transformer should transform the input into
lda = LDA(n_components=number_topics, n_jobs=-1, random_state=47)
lda.fit(count_data_tokens)

# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
companies company business market like money make capitalism buy free

Topic #1:
climate change energy oil carbon power global tax emissions nuclear

Topic #2:
removed trump report evidence russian mueller president investigation russia said

Topic #3:
people problem money need want like think stop government make

Topic #4:
china world countries country russia chinese india population europe japan

Topic #5:
people good work america job pay rich american americans money

Topic #6:
iran war military nuclear weapons trump attack iranian sanctions deal

Topic #7:
water food like use thanks plastic flag used big cancer

Topic #8:
israel land earth humans human water palestinians like israeli years

Topic #9:
trump hes vote president election party like obama republicans voting

Topic #10:
people point think thats im like make say things argument

Topic #11:
years year like people time ago old day million 10

Topic #12:
right uk left eu people party wing br

In [10]:
# # Support visual to see the separation between topic components and the label frequencies for each.
# LDAvis_prepped = sklearn_lda.prepare(lda, count_data_tokens, count_vectorizer)
# pyLDAvis.display(LDAvis_prepped)
# pyLDAvis.save_html(LDAvis_prepped, './LDAvis_prepped'+ str(number_topics) +'.html')

C:\Users\Hannah\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [11]:
# # (i) A file showing which words load on which topics

# temp = pd.DataFrame(LDAvis_prepped[2])
# temp = temp.reset_index()
# temp = temp
# #.rename(columns={4: 'Wildlife', 2: 'Water', 3: 'Photography', 1: 'Landscape', 5: 'Culture'})
# pd.crosstab(temp['Term'],temp['Topic'],values=temp['Freq'],aggfunc=sum).head()

Topic,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Term,,,,,,,,,,,,,,,,,,,,
10,0.001683,0.000579,0.004998,0.002315,0.698161,0.014362,0.002788,0.029355,0.002104,0.012415,0.119631,0.016256,0.001315,0.010259,0.04982,0.000684,0.000737,0.02057,0.011889,NaN
100f,NaN,NaN,NaN,NaN,0.987250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100k,NaN,NaN,NaN,NaN,0.960664,0.001013,0.006080,0.018240,NaN,NaN,0.001013,0.004053,0.008107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101httpwwwredditcomrhelpcomments2bx3cjreddit_101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999914
10k,NaN,NaN,NaN,NaN,0.977707,NaN,NaN,NaN,NaN,NaN,0.006837,0.005128,0.005128,NaN,NaN,NaN,NaN,NaN,0.003419,NaN


In [12]:
# len(df)

2292720